# Module 3: Creating URDF for Robots

## 🎯 Learning Objectives
- Create URDF files with links & joints
- Understand URDF → TF tree generation
- Build mobile robots using 5-step process
- Use Xacro for scalable robot descriptions

---

## 📹 Video Introduction

Watch this video before starting the module:

<div style="padding:56.25% 0 0 0;position:relative;">
  <iframe src="https://player.vimeo.com/video/YOUR_VIDEO_ID" style="position:absolute;top:0;left:0;width:100%;height:100%;" frameborder="0" allow="autoplay; fullscreen; picture-in-picture" allowfullscreen></iframe>
</div>

**Note:** Replace `YOUR_VIDEO_ID` with your actual Vimeo ID.

---

## ⚡ Creating Your First URDF

**File:** `my_robot.urdf`

```xml
<?xml version="1.0"?>
<robot name="my_robot">
    <link name="base_link">
        <visual>
            <geometry><box size="0.6 0.4 0.2" /></geometry>
            <origin xyz="0 0 0.1" rpy="0 0 0" />
        </visual>
    </link>
</robot>
```

**Visualize:**
```bash
ros2 launch urdf_tutorial display.launch.py model:=my_robot.urdf
```



![Figure 11.1](images/m3_fig1_urdf_rviz.png)

---

## 🔗 5-Step Assembly Process (Most Important!)

### 📋 Rules First:
1. Add **one** link + joint at a time
2. Never modify parent link origins
3. Verify after each change in RViz

### 🔧 Example: Add shoulder to base

**Step 1:** Add new link (origin at 0)
```xml
<link name="shoulder_link">
    <visual>
        <geometry><cylinder radius="0.1" length="0.3" /></geometry>
        <origin xyz="0 0 0" rpy="0 0 0" />
    </visual>
</link>
```

**Step 2:** Add joint (origin at 0)
```xml
<joint name="base_shoulder_joint" type="fixed">
    <parent link="base_link" />
    <child link="shoulder_link" />
    <origin xyz="0 0 0" rpy="0 0 0" />
</joint>
```

**Step 3:** Fix joint origin → DEFINES TF!
- Disable shoulder visual in RViz
- Set: `<origin xyz="0 0 0.2" rpy="0 0 0" />` (above base)

**Step 4:** Set joint type
```xml
<joint name="base_shoulder_joint" type="revolute">
    ...
    <axis xyz="0 0 1" />
    <limit lower="-3.14" upper="3.14" velocity="100" effort="100"/>
</joint>
```

**Step 5:** Fix visual origin
- Enable shoulder visual
- Set: `<origin xyz="0 0 0.15" rpy="0 0 0" />` (half cylinder length)



| Step 1-2: Initial | Step 3: Fix Joint | Step 5: Final |
|--------------------|-------------------|---------------|
| ![Step 2](images/m3_fig2_two_links.png) | ![Step 3](images/m3_fig4_joint_origin.png) | ![Step 5](images/m3_fig5_end_process.png) |

---

## 🤖 Complete Mobile Robot Example

### Design:
- Chassis: Box (0.6×0.4×0.2m)
- Wheels: Cylinders (radius 0.1m)
- Caster: Sphere (radius 0.05m)
- Base footprint: Virtual link

### Key Calculations:
```python
# Right wheel position
x = -0.15  # Behind center
y = -0.225 # Outside chassis (half chassis + half wheel)
z = 0

# Left wheel (mirror)
y = 0.225

# Caster wheel
x = 0.2    # Front
y = 0      # Centered
z = -0.05  # Below chassis
```



![Figure 11.6](images/m3_fig6_mobile_robot.png)

### TF Tree Verification:
```bash
ros2 run tf2_tools view_frames
```



![Figure 11.7](images/m3_fig7_final_tf_tree.png)

---

## ⚙️ Xacro Essentials

**Why Xacro?** Variables + Macros + Modularity

### Variables:
```xml
<xacro:property name="wheel_radius" value="0.1" />
<cylinder radius="${wheel_radius}" length="0.05" />
<origin xyz="0 0 ${base_height/2}" />
```

### Macros (Reusable):
```xml
<xacro:macro name="wheel" params="prefix x_pos">
    <link name="${prefix}_wheel_link">
        <visual>
            <geometry>
                <cylinder radius="${wheel_radius}" length="0.05" />
            </geometry>
            <origin xyz="0 0 0" rpy="${pi/2} 0 0" />
        </visual>
    </link>
    
    <joint name="${prefix}_wheel_joint" type="continuous">
        <parent link="base_link" />
        <child link="${prefix}_wheel_link" />
        <origin xyz="${x_pos} ${(base_width/2)+0.025} 0" />
        <axis xyz="0 1 0" />
    </joint>
</xacro:macro>

<xacro:wheel prefix="right" x_pos="-0.15" />
<xacro:wheel prefix="left" x_pos="-0.15" />
```

### File Organization:
```xml
<robot xmlns:xacro="http://www.ros.org/wiki/xacro">
    <xacro:include filename="properties.xacro" />
    <xacro:include filename="chassis.xacro" />
    <xacro:include filename="wheels.xacro" />
</robot>
```

---

## 🚨 Quick Reference: Joint Types

| Type | Axis | Use |
|------|------|-----|
| `fixed` | - | Static connections |
| `revolute` | `xyz="0 0 1"` | Limited rotation (arms) |
| `continuous` | `xyz="0 1 0"` | Wheels (infinite rotation) |
| `prismatic` | `xyz="1 0 0"` | Sliding motion |

---

## 🎓 Summary

In this module, you learned:

✅ How to create URDF files with links and joints  

✅ The 5-step process for assembling robot parts  

✅ How to build complete mobile robots  

✅ How to use Xacro for scalable robot descriptions  